# Homework 1 - Solução

**Aluno:** Franklin Oliveira

**Descrição:** Nesse homework, devemos criar uma aplicação web que colete dados de autores e papers do [google scholar](https://scholar.google.com.br/) e, então, exibir a informação em uma única página HTML. (vide instruções no arquivo <font color='bricks'>**hw1.pdf**</font>)

----------------

## Passo 1: Scraping

### Imports

In [1]:
import os
import sys
import bs4
import time
import zipfile
import urllib.request as request

from IPython.display import clear_output

# Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options 
from selenium.common.exceptions import NoSuchElementException

### Acessando a URL e carregando o código HTML (source)

In [2]:
# definindo caminho de pasta para o chromedriver
path = os.getcwd()

print('Your working directory is set to: {}'.format(path))
print('\nMake sure that chomedriver is at this location.')

Your working directory is set to: /media/franklin/Arquivos/EMAp/2º Trimestre/Data Science/FDS-Repo/hw1

Make sure that chomedriver is at this location.


In [3]:
# usar chromedriver para diferentes sistemas operacionais
sys.platform

'linux'

In [6]:
if 'chromedriver' not in os.listdir(path):
    print('yay')

yay


### Detectando e fazendo download do driver correto do Google Chrome 

Nessa seção, são criadas funções para detectar o driver correto do google chrome, de acordo com o sistema operacional e a versão instalada do chrome. 

<font color='red'>**OBS:** Essa parte é opcional se você já tem o driver correto do google chrome no diretório de trabalho </font>

In [5]:
def get_chrome_version(platform='unknown'):
    '''
    Função para fazer download do chromedriver correto (de acordo com a versão do chrome 
    e o sistema operacional). 
    
    OBS: Essa função funciona apenas no Jupyter Notebook. A versão "genérica" está 
    no arquivo scrape_scholar.py
    
    Argumentos:
        - platform (str): MacOS, Windows or Linux
    
    Retorna:
        - tupla: (versão_completa, nº da versão - 2 primeiros digitos)
    '''
    # ex. of url: https://chromedriver.storage.googleapis.com/74.0.3729.6/chromedriver_mac64.zip
    # platform list
    plat_list = ['macos', 'windows', 'linux']
    
    # gets platform with sys package if the user didn't input any
    if platform.lower() not in plat_list:
        print('Getting platform info with sys package...')
        platform = sys.platform
        print('Done!')
    
    if platform.lower().startswith('linux'):
        print('Getting chrome version...')
        version_full = !google-chrome --version
        version_full = version_full[0].strip().split(' ')[-1]
        print('Done!')
    
    if platform.lower().startswith('darwin') or platform.lower().startswith('mac'):
        print('Getting chrome version...')
        version_full = !/Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome --version
        print('Done!')
    
    if platform.lower().startswith('win'):
        print('Getting chrome version...')
        version_full = !reg query "HKEY_CURRENT_USER\Software\Google\Chrome\BLBeacon" /v version
        version_full = version_full[2].split(' ')[-1]
        print('Done!')
    
    time.sleep(1)
    clear_output()
    
    version_number = version_full.split('.')[0]
    print('Your chrome version is {}'.format(version_full))
    
    
    return version_full,version_number

In [6]:
# Detectando a versão do chrome
chromeVersion = get_chrome_version(platform=sys.platform)

Getting chrome version...
Done!
Your chrome version is 0


In [28]:
def download_chrome_driver(platform,version_number):
    '''
    Downloads chromedriver from https://sites.google.com/a/chromium.org/chromedriver/downloads
    
    OBS: The driver is going to be saved at your current working directory.
    
    '''
    url = 'https://chromedriver.storage.googleapis.com/{}/chromedriver_{}.zip'
    
    # exact version of chrome to generate download link
    if int(version_number) == 76:
        version2down = '76.0.3809.68'
    
    elif int(version_number) == 75:
        version2down = '75.0.3770.140'
    
    elif int(version_number) == 74:
        version2down = '74.0.3729.6'
    
    elif int(version_number) == 73:
        version2down = '73.0.3683.68'
    
    else:
        print('Check your chrome version manually and download the correct driver at:\n \
        https://sites.google.com/a/chromium.org/chromedriver/downloads')
        return None
    
    # platform info
    if platform.lower().startswith('linux'):
        try:
            request.urlretrieve(url.format(version2down,'linux64'), filename='chromedriver_linux64.zip')
            print('chromedriver downloaded at {}'.format(path))
            
            with zipfile.ZipFile('{}/{}'.format(os.getcwd(),'chromedriver_linux64.zip'), 'r') as zip_ref:
                zip_ref.extractall('{}'.format(os.getcwd()))
        except:
            print('Download failed. Please try again or access \
            https://sites.google.com/a/chromium.org/chromedriver/downloads \
            to download it manually.')
    
    elif platform.lower().startswith('darwin') or platform.lower().startswith('mac'):
        try:
            request.urlretrieve(url.format(version2down,'mac64'),filename='chromedriver_mac64.zip')
            print('chromedriver downloaded at {}'.format(path))
            
            with zipfile.ZipFile('{}/{}'.format(os.getcwd(),'chromedriver_mac64.zip'), 'r') as zip_ref:
                zip_ref.extractall('{}'.format(os.getcwd()))
        except:
            print('Download failed. Please try again or access \
            https://sites.google.com/a/chromium.org/chromedriver/downloads \
            to download it manually.')
    
    elif platform.lower().startswith('win'):
        try:
            request.urlretrieve(url.format(version2down,'win32'), filename='chromedriver_win32.zip')
            print('chromedriver downloaded at {}'.format(path))
            
            with zipfile.ZipFile('{}/{}'.format(os.getcwd(),'chromedriver_win32.zip'), 'r') as zip_ref:
                zip_ref.extractall('{}'.format(os.getcwd()))
        except:
            print('Download failed. Please try again or access \
            https://sites.google.com/a/chromium.org/chromedriver/downloads \
            to download it manually.')

In [29]:
chromeVersion[1]

'75'

In [8]:
# fazendo download do chromedriver
download_chrome_driver(sys.platform,chromeVersion[1])

chromedriver downloaded at /media/franklin/Arquivos/EMAp/2º Trimestre/Data Science/FDS-Repo/hw1


### Criando fução scrape

Essa função usa o pacote `Selenium` para navegar até o [google scholar](https://scholar.google.com.br/) e coletar as informações do autor pesquisado.

<font color='red'>**Requisitos:**</font>

> - chromedriver
> - `Selenium`
> - `sys`
> - `time`

In [25]:
def scrape(author, path_to_driver = os.getcwd()):
    
    # identifica o driver correto com base no sistema operacional
    # inicializa o driver
    chrome_options = Options()  
    #chrome_options.add_argument("--headless")    
    
    if sys.platform.lower() == 'windows':
        driver = webdriver.Chrome('{}/chromedriver.exe'.format(path_to_driver), 
                                  chrome_options = chrome_options)
    else:
        driver = webdriver.Chrome('{}/chromedriver'.format(path_to_driver), 
                                  chrome_options = chrome_options)

    # acessa URL
    # https://scholar.google.com.br/scholar?hl=EN&as_sdt=0,5&q=renato+rocha+souza
    driver.get("https://scholar.google.com.br/")
    time.sleep(0.5)

    # encontra campo de busca
    search_term = driver.find_element_by_id('gs_hdr_tsi')

    try:
        # input para pesquisa
        search_term.clear() # limpa a caixa de texto
        search_term.send_keys("{}".format(author))  # insere elemento de busca
        search_term.send_keys(Keys.RETURN) # envia o termo de busca

        # clica no botão de busca
        search_btn = driver.find_element_by_class_name('gs_rt') 
        res = search_btn.find_element_by_tag_name('a') 
        res.click()
        time.sleep(0.3)
        
        # clica no primeiro autor (resultado da busca)
        res_photo = driver.find_element_by_class_name('gs_ai_pho')
        res_photo.click()


        # scrolling and loading more
        SCROLL_PAUSE_TIME = 0.3

        # Altura do scroll
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            show_more = driver.find_element_by_id('gsc_bpf_more')
            show_more.click()

            # Espera para carregar a página
            time.sleep(SCROLL_PAUSE_TIME)

            # Calcula a nova altura de Scroll e compara com a última
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            
            time.sleep(SCROLL_PAUSE_TIME)

        # url + source code (totalmente carregado)
        full_url = driver.current_url
        source = driver.page_source

        print('Dados coletados. Verifique código html na variável source.')

    except NoSuchElementException as E:
        print('Autor não encontrado. Verifique se o nome pesquisado está correto.')
        print('Fechando driver...')
        full_url = driver.current_url
        source = ''

    driver.quit()
    
    # returns a tuple
    return full_url, source

In [26]:
data = scrape('Jeffrey Heer')
source = data[1]

/home/franklin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


Dados coletados. Verifique código html na variável source.


### Coletando informações de papers e autores

Vamos buscar as publicações (títulos dos artigos e autores) no código html da página do google scholar.

In [55]:
def find_papers_and_authors(scholar_source):
    
    # html coletada do google scholar
    html = bs4.BeautifulSoup(scholar_source)
    
    # tags com a informação que queremos (artigos e autores)
    attr = html.find_all('tr')
    
    # coletando autores
    authors = [str(tag.find('div')).split('>')[1].split('<')[0].replace(', ...','').strip() for 
           tag in attr[6:]]
    
    # ajustando formato (autor:[aut1,aut2,aut3])
    res = []
    for lista in [autor.split(',') for autor in authors]:
        res_1 = []
        for i in lista:
            res_1.append(i.strip())
        res.append(res_1)
    authors = res
    
    # coletando papers (obs: tratando para remover <i> some_text </i> de alguns títulos)
    works = [str(tag.find('a')).replace('<i>','').replace('</i>','').split('">')[1].split('</')[0]
          for tag in attr[6:]]
    
    papers = [{'titulo':work , 'autores':author} for (work,author) in zip(works,authors)]
    
    return papers

In [54]:
find_papers_and_authors(source)

[{'titulo': 'D3: Data-Driven Documents',
  'autores': ['M Bostock', 'V Ogievetsky', 'J Heer']},
 {'titulo': 'Prefuse: a toolkit for interactive information visualization',
  'autores': ['J Heer', 'SK Card', 'JA Landay']},
 {'titulo': 'Vizster: Visualizing online social networks',
  'autores': ['J Heer', 'D Boyd']},
 {'titulo': 'Narrative visualization: Telling stories with data',
  'autores': ['E Segel', 'J Heer']},
 {'titulo': 'Crowdsourcing graphical perception: using mechanical turk to assess visualization design',
  'autores': ['J Heer', 'M Bostock']},
 {'titulo': 'Interactive dynamics for visual analysis',
  'autores': ['J Heer', 'B Shneiderman']},
 {'titulo': 'A tour through the visualization zoo.',
  'autores': ['J Heer', 'M Bostock', 'V Ogievetsky']},
 {'titulo': 'Animated transitions in statistical data graphics',
  'autores': ['J Heer', 'G Robertson']},
 {'titulo': 'Profiles as conversation: Networked identity performance on Friendster',
  'autores': ['D Boyd', 'J Heer']},
 {

<br>

**Fim do Notebook**